In [16]:
import extract_data as dt
import numpy as np
import simple_unet as ml
import torch
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import os
import ast
import pandas as pd

ERROR: Could not find a version that satisfies the requirement open3d (from versions: none)
ERROR: No matching distribution found for open3d


ModuleNotFoundError: No module named 'open3d'

## Data
First we prepare our training data. To do this, we read in the ultrasound `.raw` scans and extract the point data from them and store them as vertices. We do the same for the 5 labelled `.ply` files.

In [14]:
folder_path = "../../volumes"

# Get a list of all .raw files in the folder
training_data = [f for f in os.listdir(folder_path) if f.endswith('.raw')]

# Loop through each raw file, load it, reshape it, and extract the points and vertices
for file_name in raw_files:
    file_path = os.path.join(folder_path, file_name)

    # Load the raw image data
    raw_img = np.fromfile(file_path, dtype=np.dtype('<u2'))

    # Reshape the image (assuming all files have the same shape)
    raw_img = np.reshape(raw_img, (1280, 768, 768)).astype(np.float32)

    # Extract point data using the provided function
    point_data = dt.extract_point_data(raw_img)

    # Extract vertices from the point data
    vertices = dt.extract_vertices(point_data)

    # Print the point data and vertices for the first 5 points
    print(f"Processing file: {file_name}")
    print("Point Data (first 5 points):\n", point_data[:5])
    print("Vertices (first 5 points):\n", vertices[:5])


NameError: name 'os' is not defined

In [5]:
vertices_str = np.array2string(vertices, separator=',', formatter={'float_kind': lambda x: str(x).replace('.', ',')})
print(vertices_str)

NameError: name 'np' is not defined

In [6]:
folder_path = "../../meshes"

# Get a list of all .ply files in the folder
ply_files = [f for f in os.listdir(folder_path) if f.endswith('.ply')]

# Create a list to store the mesh data
all_mesh_data = []

# Loop through each ply file, extract the mesh data, and store it
for file_name in ply_files:
    file_path = os.path.join(folder_path, file_name)

    # Extract mesh data from the .ply file
    mesh_data = dt.extract_ply_o3(file_path)

    # Add the mesh data to the list if it was successfully read
    if mesh_data is not None:
        all_mesh_data.append(mesh_data)

# Convert the list of mesh data into a NumPy array, if desired
if all_mesh_data:
    ans_data = np.array(all_mesh_data)

    # Display the extracted data for verification
    print("Extracted Mesh Data (first mesh):\n", ans_data[0])

NameError: name 'os' is not defined

In [7]:
# Here we load the scan and reshape them, as per the Discord example
raw_img = np.fromfile("../../volumes/scan_005.raw", dtype=np.dtype('<u2'))
raw_img = np.reshape(np.array(raw_img), (1280, 768, 768)).astype(np.float32)

NameError: name 'np' is not defined

In [8]:
# get vertices data, begin building our model inputs
point_data = dt.extract_point_data(raw_img)
vertices = dt.extract_vertices(point_data)
print("Point Data (first 5 points):\n", point_data[:5])
print("Vertices (first 5 points):\n", vertices[:5])

NameError: name 'dt' is not defined

In [9]:
ans_data = dt.extract_ply("../../meshes/scan_005.ply")
ans_data

NameError: name 'dt' is not defined

In [26]:
input_grid = np.zeros((64, 64, 64))
answer_grid = np.zeros((64, 64, 64))

for vertex in vertices:
    x, y, z = vertex.astype(int)
    x, y, z = np.clip([x, y, z], 0, 63)
    input_grid[x, y, z] = 1  # set val on grid

for point in ans_data:
    x, y, z = point.astype(int)
    x, y, z = np.clip([x, y, z], 0, 63)
    answer_grid[x, y, z] = 1 

input_grid
answer_grid

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

Now we convert our inputs and answers into tensors.

In [27]:
input_tensor = torch.tensor(input_grid, dtype=torch.float32).unsqueeze(0).unsqueeze(0)  
answer_tensor = torch.tensor(answer_grid, dtype=torch.float32).unsqueeze(0).unsqueeze(0) 

In [28]:
model = ml.SimpleUNet3D()
output = model(input_tensor)

In [12]:
criterion = torch.nn.MSELoss()
loss = criterion(output, answer_tensor)
print(f"Loss: {loss.item()}")

Loss: 0.2945975065231323


In [30]:
num_epochs = 1

for epoch in range(num_epochs):
    print(f"Starting epoch {epoch + 1}/{num_epochs}")
    for i, (inputs, targets) in enumerate(dataloader):
        print(f"Batch {i + 1}, inputs shape: {inputs.shape}, targets shape: {targets.shape}")
        # Forward pass
        outputs = model(inputs)
        # Compute the loss
        loss = criterion(outputs, targets)
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

print("Training complete.")

Starting epoch 1/1


NameError: name 'dataloader' is not defined